In [202]:
# https://pstu.ru/sveden/education/
# https://pstu.ru/activity/educational/high/fgos/
# url = input()
url = 'https://pstu.ru/activity/educational/high/fgos/'
url

'https://pstu.ru/activity/educational/high/fgos/'

In [203]:
import os
from urllib.parse import urlparse

def make_root_dir_path(url : str, base_url:str=None) -> str:
    if not base_url:
        curdir = os.path.abspath(os.curdir)
    else:
        curdir = make_root_dir_path(base_url)
    parsed_url = urlparse(url)
    domen = parsed_url.netloc.replace('.', '').replace('/', '')
    path = parsed_url.path.replace('/', '')
    dir_name = domen+path
    dir_path = os.path.join(curdir, dir_name)

    return dir_path

# make_root_dir_path('https://abcd.local/boring','https://abcd.local')

def make_root_dir(url : str, base_url:str=None) -> str:
    path = make_root_dir_path(url, base_url)

    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [204]:
import urllib.request
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import shutil
# from urllib.parse import quote_plus

def request_not_pdf_links(url : str) -> list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if not href.lower().endswith('.pdf'):
            links.append(urljoin(url, href))
    return list(set(links))

def request_pdf_links(url : str) -> list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        lower_href = href.lower()
        if lower_href.endswith('.pdf'):
            links.append(href)
    return list(set(links))

def download_pdfs(url : str, loaded_files : set, base_url:str=None) -> list:
    page_file_path = make_root_dir(url, base_url)
    file_links = request_pdf_links(url)
    parsed_url = urlparse(url)
    root_url = '://'.join([ parsed_url.scheme, parsed_url.netloc ])
    file_pathes = []
    for file_link in file_links:
        if os.path.basename(file_link) in loaded_files:
            continue
        file_url = urljoin(root_url, file_link.replace(' ', '%20')).encode().decode()
        file_path = os.path.join(page_file_path, os.path.basename(file_link))
        if not os.path.exists(file_path):
            response = requests.get(file_url)
            content_type = response.headers.get('content-type')
            if response.status_code == 200 and content_type == 'application/pdf':
                file_content = response.content
                with open(file_path, 'wb') as file:
                    file.write(file_content)
            else:
                continue
        file_pathes.append(file_path)
    return list(set(file_pathes))

# download_pdfs(url)

In [205]:
# import requests
# import os
# content = requests.get('https://pstu.ru/files/file/Abitur/%D0%91%D0%A3%D0%9F%D1%8B/2019/%7B8b61fce0-770b-443b-a572-f3b72fe8f412%7D.pdf').content
# os.makedirs('./tmp/')
# with open('./tmp/test.pdf', 'wb') as file:
#     file.write(content)

In [206]:
# !sudo dnf install ocrmypdf tesseract-osd 
# !sudo dnf install tesseract-langpack-rus
# !pip3 install ocrmypdf

In [207]:
from pypdf import PdfReader
import ocrmypdf

def get_pdf_reader_text(pdf_reader: PdfReader) -> str:
    text = str()
    for page in pdf_reader.pages:
        text += '\n' + (page.extract_text())
    return text

def extract_text_from_pdf(pdf_path : str):
    try:
        reader = PdfReader(pdf_path)
        if not reader.metadata:
            return
        text = get_pdf_reader_text(reader)
        if text.strip():
            return text
    except:
        pass
    ocrmypdf.ocr(pdf_path, pdf_path, language=['rus','eng'], optimize=1, redo_ocr=True)
    reader = PdfReader(pdf_path)    
    return get_pdf_reader_text(reader)

def save_text_from_pdf(pdf_path : str):
    text = extract_text_from_pdf(pdf_path)
    dir_name = os.path.dirname(pdf_path)
    file_name = os.path.basename(pdf_path)
    file_name_no_extention = os.path.splitext(file_name)[0]
    target_path = os.path.join(dir_name, file_name_no_extention + '.txt')
    
    with open(target_path, 'wb') as file:
        try:
            file.write(text.encode())
            print(f'completed: {target_path}')
        except:
            print(f'Failed: {target_path}')
            pass
        finally:
            file.close()

def load_pdf_and_text(url : str, loaded_files : set, base_url:str = None) -> list:
    file_pathes = download_pdfs(url, loaded_files, base_url)
    for file_path in file_pathes:
        save_text_from_pdf(file_path)
    return file_pathes

# load_pdf_and_text(url)



In [208]:
import tarfile
import shutil

def compress_folder_to_file(output_filename : str, source_dir : str, remove_source_dir=False):
    if os.path.exists(output_filename):
        os.remove(output_filename)
    with tarfile.open(output_filename, 'w:gz') as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir), recursive=True)
    
    if remove_source_dir:
        shutil.rmtree(source_dir, ignore_errors=True)

# dir_path = make_root_dir_path(url)
# compress_folder_to_file(dir_path + '.tar.gz', dir_path)

In [212]:
max_level = 2
loaded_files = set()

def traverse(url : str, level = 0, base_url=None) -> None:
    if level >= max_level:
        return
    for loaded_file in load_pdf_and_text(url, loaded_files, base_url):
        loaded_files.add(os.path.basename(loaded_file))

    not_file_link = request_not_pdf_links(url)
    for link in not_file_link:
        traverse(link, level + 1, url)

    if level == 0:
        dir_path = make_root_dir_path(url)
        compress_folder_to_file(dir_path + '.tar.gz', dir_path)

traverse(url)

In [213]:
max_level = 3
loaded_files = set()
url = 'https://pstu.ru/sveden/education/'

traverse(url)


Output()

Output()

[tesseract] lots of diacritics - possibly poor OCR

[tesseract] lots of diacritics - possibly poor OCR

[tesseract] lots of diacritics - possibly poor OCR

...

Output()

Output()

completed: /home/lolomarka/Documents/web-analysis/lab2/psturusvedeneducation/{cc4a0b89-7a3d-41ee-871e-ed26911f5cfb}.txt


Output()

InputFileError: pdfminer could not process page 0 (counting from 0).